In [3]:
import copy
import math
import tqdm
import numpy
import torch
import torch.nn as nn
import torch.utils.data as Data
from random import *
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

batch_size = 64
src_vocab_size = 21128          # 字典大小
d_model = 512                   # embedding维度
num_layers = 2                  # LSTM层数
hidden_size = 100               # LSTM隐藏层
linear_hidden_size = 10         # 全链接层隐藏数
classes = 2                     # 类别数
dropout = 0.3                   # LSTM中dropout
lr = 1e-3                       # 学习率
epochs = 10                     # 训练次数   

In [4]:
class myNet(torch.nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.embed = torch.nn.Embedding(src_vocab_size, d_model)                        
        self.lstm = torch.nn.LSTM(input_size=d_model, hidden_size=hidden_size,num_layers=num_layers,dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, linear_hidden_size)
        self.linear1 = torch.nn.Linear(linear_hidden_size, classes)
    def forward(self,data):
        x = self.embed(data)                             # [64,32,512]   把字用字向量表示
        x,(h_n, c_n) = self.lstm(x.transpose(0, 1))      # x:[32, 64, 100]    记录每时刻最后一层的输出。
                                                         # h_n: [2, 64, 100]  记录每一层最后一次的输出
                                                         # c_n: [2, 64, 100]  记录每一层cell保存的值
        x = self.linear(x[-1])                           # [64, 10] 经过第一层全连接层
        x = self.linear1(x)                              # [64, 2] 经过第二层全连接层
        return x       

model = myNet().to(device)

In [5]:
model.load_state_dict(torch.load('model.pt', map_location=torch.device('cpu')))
for name, parameters in model.named_parameters():
    print(name, ':', parameters.size())

embed.weight : torch.Size([21128, 512])
lstm.weight_ih_l0 : torch.Size([400, 512])
lstm.weight_hh_l0 : torch.Size([400, 100])
lstm.bias_ih_l0 : torch.Size([400])
lstm.bias_hh_l0 : torch.Size([400])
lstm.weight_ih_l1 : torch.Size([400, 100])
lstm.weight_hh_l1 : torch.Size([400, 100])
lstm.bias_ih_l1 : torch.Size([400])
lstm.bias_hh_l1 : torch.Size([400])
linear.weight : torch.Size([10, 100])
linear.bias : torch.Size([10])
linear1.weight : torch.Size([2, 10])
linear1.bias : torch.Size([2])


In [6]:
import openvino as ov
ov_model = ov.convert_model('lstm_model.onnx')
ov.save_model(ov_model, 'model.xml')

ModuleNotFoundError: No module named 'openvino'

In [7]:
from openvino.runtime import Core
# 实例化Core对象
core = Core() 
det_ov_model = core.read_model('model.xml')
    
net = core.compile_model(det_ov_model, device_name="AUTO")

ModuleNotFoundError: No module named 'openvino'

In [8]:
pad_size = 60
with open("./vocab.txt", 'r', encoding='UTF-8') as f:
        idx2word = {idx: line.strip() for idx, line in  enumerate(tqdm.tqdm(f))}
        word2idx = {idx2word[key]: key for key in  idx2word}
content = '(长期诚信在本市作各类资格职称以及印章、牌等。电话：11451419198 李伟'
token_ids = []
token_ids.append (word2idx['[CLS]'])
for key in content:
    token_ids.append(word2idx.get(key, 0))
seq_len = len(token_ids)
mask = []
if pad_size:
    if seq_len < pad_size:
        token_ids += ([0] * (pad_size - seq_len))
    else:
        token_ids = token_ids[:pad_size]
        seq_len = pad_size
content
token_ids
datain = torch.tensor(token_ids)
datain = datain.unsqueeze(0)
datain

21128it [00:00, 3638714.58it/s]


tensor([[ 101,  113, 7270, 3309, 6411,  928, 1762, 3315, 2356,  868, 1392, 5102,
         6598, 3419, 5466, 4917,  809, 1350, 1313, 4995,  510, 4277, 5023,  511,
         4510, 6413, 8038,  122,  122,  125,  126,  122,  125,  122,  130,  122,
          130,  129,    0, 3330,  836,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])

In [9]:
import time

start_time_torch = time.time()
test_output = model(datain.to(device)) 
end_time_torch = time.time()

inference_time_torch = end_time_torch - start_time_torch
inference_speed_torch = 1 / inference_time_torch

print(test_output.argmax(dim=1))
print(test_output)

print("使用PyTorch推理时间：", inference_time_torch, "秒")
print("使用PyTorch推理速度：", inference_speed_torch, "次/秒")

tensor([1])
tensor([[-4.7629,  1.0060]], grad_fn=<AddmmBackward0>)
使用PyTorch推理时间： 0.07451820373535156 秒
使用PyTorch推理速度： 13.419539788579181 次/秒


In [10]:
def get_response(msg):
    from openvino.runtime import Core
    # 实例化Core对象
    core = Core() 
    det_ov_model = core.read_model('model.xml')
        
    net = core.compile_model(det_ov_model, device_name="AUTO")
    pad_size = 60
    with open("./vocab.txt", 'r', encoding='UTF-8') as f:
            idx2word = {idx: line.strip() for idx, line in  enumerate(tqdm.tqdm(f))}
            word2idx = {idx2word[key]: key for key in  idx2word}
    content = str(msg)
    token_ids = []
    token_ids.append (word2idx['[CLS]'])
    for key in content:
        token_ids.append(word2idx.get(key, 0))
    seq_len = len(token_ids)
    mask = []
    if pad_size:
        if seq_len < pad_size:
            token_ids += ([0] * (pad_size - seq_len))
        else:
            token_ids = token_ids[:pad_size]
            seq_len = pad_size
    content
    token_ids
    datain = torch.tensor(token_ids)
    datain = datain.unsqueeze(0)
    output_node = net.outputs[0] 
    ir = net.create_infer_request()
    outputs = ir.infer(datain)[output_node]
    if outputs.argmax() == 1:
        return "这是垃圾内容"
    else:
        return "这是安全内容"

In [11]:
output_node = net.outputs[0] 
ir = net.create_infer_request()

start_time_openvino = time.time()
outputs = ir.infer(datain)[output_node]
end_time_openvino = time.time()

inference_time_openvino = end_time_openvino - start_time_openvino
inference_speed_openvino = 1 / inference_time_openvino

print("使用OpenVINO推理时间：", inference_time_openvino, "秒")
print("使用OpenVINO推理速度：", inference_speed_openvino, "次/秒")
print(outputs.argmax())

NameError: name 'net' is not defined

In [ ]:
from flask import Flask, render_template, request
import tqdm
import copy
import math
import tqdm
import numpy
import torch
import torch.nn as nn
import torch.utils.data as Data
from random import *
app = Flask(__name__)


@app.route("/")
def home():    
    return render_template("./index.html")
@app.route("/get")
def get_bot_response():    
    userText = request.args.get('msg')  
    response = str(get_response(userText))
    return response
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
import gradio as gr
import tqdm
import copy
import math
import tqdm
import numpy
import torch
import torch.nn as nn
import torch.utils.data as Data
from random import *

with gr.Blocks() as demo:
    gr.Markdown(
    """
    # 信息内容检测器
    在输入块中键入内容查看结果！
    """)
    name = gr.Textbox(label="输入")
    output = gr.Textbox(label="判断")
    submit_btn = gr.Button("提交")
    submit_btn.click(fn=get_response, inputs=name, outputs=output, api_name="get_response")
demo.launch()   